# 0005.0003 Modeling Strategy - Betting, Expected Values

In [1]:
import pathlib
import os
import sys

%matplotlib inline
import matplotlib.pyplot as plt # creating visualizations
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import pandas as pd
import numpy as np # basic math and random numbers

import torch # package for building functions with learnable parameters
import torch.nn as nn # prebuilt functions specific to neural networks
from torch.autograd import Variable # storing data while learning

from scipy import optimize
from sklearn.metrics import mean_squared_error

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
# import my class code from the source
# %aimport src-dir.filename

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


DATA_PATH = pathlib.Path().cwd().resolve().parent / 'data' / 'soccer' / 'raw' / 'United Kingdom' / 'english premier league' / '2000-2001' / 'football-data-co-uk' / 'season-data' / '2000-2001.csv' 
print(DATA_PATH)

/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/soccer/raw/United Kingdom/english premier league/2000-2001/football-data-co-uk/season-data/2000-2001.csv


In [2]:
df = pd.read_csv(DATA_PATH)
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Attendance,Referee,HS,AS,HST,AST,HHW,AHW,HC,AC,HF,AF,HO,AO,HY,AY,HR,AR,HBP,ABP,GBH,GBD,GBA,IWH,IWD,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA
0,E0,19/08/00,Charlton,Man City,4,0,H,2,0,H,20043,Rob Harris,17,8,14,4,2,1,6,6,13,12,8,6,1,2,0,0,10,20,2.00,3.0,3.2,2.20,2.9,2.7,2.20,3.25,2.75,2.20,3.25,2.88,2.10,3.2,3.10
1,E0,19/08/00,Chelsea,West Ham,4,2,H,1,0,H,34914,Graham Barber,17,12,10,5,1,0,7,7,19,14,2,3,1,2,0,0,10,20,1.47,3.4,5.2,1.60,3.2,4.2,1.50,3.40,6.00,1.50,3.60,6.00,1.44,3.6,6.50
2,E0,19/08/00,Coventry,Middlesbrough,1,3,A,1,1,D,20624,Barry Knight,6,16,3,9,0,1,8,4,15,21,1,3,5,3,1,0,75,30,2.15,3.0,3.0,2.20,2.9,2.7,2.25,3.20,2.75,2.30,3.20,2.75,2.30,3.2,2.62
3,E0,19/08/00,Derby,Southampton,2,2,D,1,2,A,27223,Andy D'Urso,6,13,4,6,0,0,5,8,11,13,0,2,1,1,0,0,10,10,2.00,3.1,3.2,1.80,3.0,3.5,2.20,3.25,2.75,2.05,3.20,3.20,2.00,3.2,3.20
4,E0,19/08/00,Leeds,Everton,2,0,H,2,0,H,40010,Dermot Gallagher,17,12,8,6,0,0,6,4,21,20,6,1,1,3,0,0,10,30,1.65,3.3,4.3,1.55,3.3,4.5,1.55,3.50,5.00,1.57,3.60,5.00,1.61,3.5,4.50


In [3]:
df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA']]
df.describe()

,FTHG,FTAG,LBH,LBD,LBA,WHH,WHD,WHA
count,380.000000,380.000000,325.000000,325.000000,325.000000,380.000000,380.000000,380.000000
mean,1.544737,1.065789,2.190277,3.415569,3.967354,2.182237,3.403079,4.009053
std,1.285436,1.029070,0.888833,0.347241,2.086878,0.919950,0.378054,2.139420
min,0.000000,0.000000,1.170000,2.870000,1.360000,1.160000,2.900000,1.330000
25%,1.000000,0.000000,1.570000,3.200000,2.400000,1.570000,3.200000,2.500000
50%,1.000000,1.000000,1.910000,3.250000,3.250000,1.900000,3.300000,3.400000
75%,2.000000,2.000000,2.500000,3.500000,5.000000,2.500000,3.500000,5.000000
max,6.000000,4.000000,7.000000,5.500000,12.000000,8.000000,5.500000,12.000000


## Betting

#### Decimal Odds:

Take decimal odds of 2.5 - Bet 1 stake, and if a win will get 2.5 back - the original stake + profit of $1.5

If the bet loses, then lose $1

#### Overround:

If home win, draw, and away win have odds of 2, 3.25, 3.4, the overround will be 
$$(\frac{1}{2} + \frac{1}{3.25} + \frac{1}{3.4}) \times 100\% = 110.2\%$$

The more outcomes in an event, the more the overround, because there is more uncertainty in the eventual outcome

#### Expected Value:

Make a bet on Man City to win a game at decimal odds of 3.5 - `o`

We believe the true chance of a win is 0.25 (25%) - `p`

We stake $1 `s`

$$Expected\; Value = p \times o - 1$$

$$EV = 0.25 \times 3.5 -1 = -0.125$$


#### Return

$$Expected\; Return = p \times o$$ 

$$Expected\; Return = 0.25 \times 3.5 = 0.875$$

that is after betting 1 we expect to have $0.875

#### Fair Odds

fair odds - $f = \frac{1}{p}$

Where $


Where $o > f$, $R > 1$ we should place the bet 


## Simulated EV, and Returns


We will simulate some probabilities as though we have generated them from a model, and the simulate some bets

In [4]:
# Simulate Some Model Outputs
df['p_hwin'] = np.random.uniform(0.1, 0.4, size=len(df))
df['p_awin'] = np.random.uniform(0.05, 0.4, size=len(df))
df['p_draw'] = 1 - (df['p_hwin'] + df['p_awin'])
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LBH,LBD,LBA,WHH,WHD,WHA,p_hwin,p_awin,p_draw
0,19/08/00,Charlton,Man City,4,0,H,2.20,3.25,2.75,2.10,3.2,3.10,0.236714,0.117019,0.646267
1,19/08/00,Chelsea,West Ham,4,2,H,1.50,3.40,6.00,1.44,3.6,6.50,0.217062,0.244440,0.538499
2,19/08/00,Coventry,Middlesbrough,1,3,A,2.25,3.20,2.75,2.30,3.2,2.62,0.120943,0.069644,0.809413
3,19/08/00,Derby,Southampton,2,2,D,2.20,3.25,2.75,2.00,3.2,3.20,0.291896,0.062182,0.645922
4,19/08/00,Leeds,Everton,2,0,H,1.55,3.50,5.00,1.61,3.5,4.50,0.233271,0.109356,0.657372


In [5]:
# calculate the EVs
df['EV_hwin'] = (df['p_hwin'] * df['WHH']) - 1
df['EV_awin'] = (df['p_awin'] * df['WHA']) - 1
df['EV_draw'] = (df['p_draw'] * df['WHD']) - 1
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LBH,LBD,LBA,WHH,WHD,WHA,p_hwin,p_awin,p_draw,EV_hwin,EV_awin,EV_draw
0,19/08/00,Charlton,Man City,4,0,H,2.20,3.25,2.75,2.10,3.2,3.10,0.236714,0.117019,0.646267,-0.502900,-0.637242,1.068054
1,19/08/00,Chelsea,West Ham,4,2,H,1.50,3.40,6.00,1.44,3.6,6.50,0.217062,0.244440,0.538499,-0.687431,0.588858,0.938595
2,19/08/00,Coventry,Middlesbrough,1,3,A,2.25,3.20,2.75,2.30,3.2,2.62,0.120943,0.069644,0.809413,-0.721832,-0.817533,1.590122
3,19/08/00,Derby,Southampton,2,2,D,2.20,3.25,2.75,2.00,3.2,3.20,0.291896,0.062182,0.645922,-0.416208,-0.801016,1.066950
4,19/08/00,Leeds,Everton,2,0,H,1.55,3.50,5.00,1.61,3.5,4.50,0.233271,0.109356,0.657372,-0.624433,-0.507896,1.300802


In [6]:
# Bet on positive EV, and calculate profits/losses
df1 = df.copy(deep=True)
df1 = df1[['HomeTeam', 'AwayTeam', 'FTR', 'WHH', 'WHD', 'WHA', 'EV_hwin', 'EV_awin', 'EV_draw',
          'p_hwin', 'p_awin', 'p_draw']]
EV_threshold = 0.0

for bet in ['hwin', 'draw', 'awin']:
    betl = bet[0].capitalize()
    bets = (df1['EV_' + bet] > EV_threshold)
    df1.loc[bets, 'PROFIT_' + betl] = np.where(df1.loc[bets, 'FTR'] == betl, df1.loc[bets,'WH'+ betl]-1, -1)
    
df1.head(20)

,HomeTeam,AwayTeam,FTR,WHH,WHD,WHA,EV_hwin,EV_awin,EV_draw,p_hwin,p_awin,p_draw,PROFIT_H,PROFIT_D,PROFIT_A
0,Charlton,Man City,H,2.10,3.20,3.10,-0.502900,-0.637242,1.068054,0.236714,0.117019,0.646267,NaN,-1.00,NaN
1,Chelsea,West Ham,H,1.44,3.60,6.50,-0.687431,0.588858,0.938595,0.217062,0.244440,0.538499,NaN,-1.00,-1.00
2,Coventry,Middlesbrough,A,2.30,3.20,2.62,-0.721832,-0.817533,1.590122,0.120943,0.069644,0.809413,NaN,-1.00,NaN
3,Derby,Southampton,D,2.00,3.20,3.20,-0.416208,-0.801016,1.066950,0.291896,0.062182,0.645922,NaN,2.20,NaN
4,Leeds,Everton,H,1.61,3.50,4.50,-0.624433,-0.507896,1.300802,0.233271,0.109356,0.657372,NaN,-1.00,NaN
5,Leicester,Aston Villa,D,2.40,3.25,2.50,-0.367821,-0.457287,0.688396,0.263408,0.217085,0.519507,NaN,2.25,NaN
6,Liverpool,Bradford,H,1.33,4.00,8.00,-0.655810,0.046468,1.441609,0.258789,0.130808,0.610402,NaN,-1.00,-1.00
7,Sunderland,Arsenal,H,3.75,3.00,1.90,-0.107461,-0.623970,0.692237,0.238010,0.197911,0.564079,NaN,-1.00,NaN
8,Tottenham,Ipswich,H,1.44,3.60,6.50,-0.494695,0.282528,0.626414,0.350906,0.197312,0.451782,NaN,-1.00,-1.00
9,Man United,Newcastle,H,1.40,3.75,7.00,-0.530965,1.411274,0.201903,0.335025,0.344468,0.320508,NaN,-1.00,-1.00


In [7]:
n_bets = df1[['PROFIT_H', 'PROFIT_D', 'PROFIT_A']].count()
n_bets

PROFIT_H     23
PROFIT_D    359
PROFIT_A    119
dtype: int64

In [8]:
rets = df1[['PROFIT_H', 'PROFIT_D', 'PROFIT_A']].sum()
rets

PROFIT_H     1.85
PROFIT_D   -42.00
PROFIT_A   -38.29
dtype: float64

In [9]:
ret = df1[['PROFIT_H', 'PROFIT_D', 'PROFIT_A']].sum().sum()
ret

-78.44

## References

https://www.amazon.ca/How-Find-Black-Coal-Cellar/dp/1843440679